In [1]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install --quiet -v --no-cache-dir ./

Cloning into 'apex'...
remote: Enumerating objects: 8054, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 8054 (delta 68), reused 97 (delta 44), pack-reused 7913
Receiving objects: 100% (8054/8054), 14.11 MiB | 26.96 MiB/s, done.
Resolving deltas: 100% (5467/5467), done.
/content/apex
Processing /content/apex
  Created wheel for apex: filename=apex-0.1-cp37-none-any.whl size=204709 sha256=a532fb26c1bcef2c3e2b7d49101e9cf04c4ea76d175870c556a2e8550f22e4b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-8om69unv/wheels/b1/3a/aa/d84906eaab780ae580c7a5686a33bf2820d8590ac3b60d5967
Successfully built apex


In [2]:
cd ..

/content


In [3]:
!pip install -U PyYAML

     |████████████████████████████████| 645kB 7.1MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!git clone -b feature/DR-images https://github.com/jmarrietar/suncet.git

Cloning into 'suncet'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 231 (delta 127), reused 179 (delta 82), pack-reused 0
Receiving objects: 100% (231/231), 1.09 MiB | 18.61 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [5]:
cd suncet

/content/suncet


In [6]:
!mkdir datasets

In [7]:
!mkdir datasets/dr

In [8]:
!mkdir logs

In [9]:
!python download.py -d sample@1000

Downloading...
From: https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2
To: /content/suncet/datasets/dr/sample@1000.zip
108MB [00:00, 125MB/s]  


In [10]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

import os

# -- FOR DISTRIBUTED TRAINING ENSURE ONLY 1 DEVICE VISIBLE PER PROCESS
try:
    # -- WARNING: IF DOING DISTRIBUTED TRAINING ON A NON-SLURM CLUSTER, MAKE
    # --          SURE TO UPDATE THIS TO GET LOCAL-RANK ON NODE, OR ENSURE
    # --          THAT YOUR JOBS ARE LAUNCHED WITH ONLY 1 DEVICE VISIBLE
    # --          TO EACH PROCESS
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['SLURM_LOCALID']
except Exception:
    pass

import logging
import sys
from collections import OrderedDict

import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter
)
from src.losses import (
    init_simclr_loss,
    init_suncet_loss,
    make_labels_matrix
)
from src.data_manager import (
    init_data,
    make_transforms
)
from src.sgd import SGD
from src.lars import LARS

import apex
from torch.nn.parallel import DistributedDataParallel
import argparse

import torch.multiprocessing as mp

import pprint
import yaml


In [13]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')
parser.add_argument(
    '--devices', type=str, nargs='+', default=['cuda:0'],
    help='which devices to use on local machine')
parser.add_argument(
    '--sel', type=str,
    help='which script to run',
    choices=[
        'paws_train',
        'suncet_train',
        'fine_tune',
        'snn_fine_tune'
    ])

_StoreAction(option_strings=['--sel'], dest='sel', nargs=None, const=None, default=None, type=<class 'str'>, choices=['paws_train', 'suncet_train', 'fine_tune', 'snn_fine_tune'], help='which script to run', metavar=None)

In [14]:
args = parser.parse_args(['--sel', 'paws_train',
                          '--fname', 'configs/paws/dr_train.yaml'
])

In [15]:
fname = args.fname
sel = args.sel

In [16]:
import logging
logging.basicConfig()
logger = logging.getLogger()

logger.info(f'called-params {sel} {fname}')

INFO:root:called-params paws_train configs/paws/dr_train.yaml


In [17]:
rank = 0

In [18]:
# -- load script params
params = None
with open(fname, 'r') as y_file:
    params = yaml.load(y_file, Loader=yaml.FullLoader)
    logger.info('loaded params...')
    if rank == 0:
        pp = pprint.PrettyPrinter(indent=4)
        pp.pprint(params)

if rank == 0:
    dump = os.path.join(params['logging']['folder'], f'params-{sel}.yaml')
    with open(dump, 'w') as f:
        yaml.dump(params, f)

INFO:root:loaded params...
{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_imgs_per_class': 32,
                     'supervised_views': 1,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 32},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'image_folder': 'dr/sample@1000/',
                'label_smoothing': 0.1,
                'multicrop': 6,
                'normalize': True,
                'root_path': 'datasets/',
                'subset_path': 'dr_subsets',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
                'device': 'cuda:0',
                'load_checkpoint': False,
                'model_name': 'resnet50',


In [19]:
args = params

In [20]:
def init_model(
    device,
    model_name='resnet50',
    use_pred=False,
    output_dim=128
):
    if 'wide_resnet' in model_name:
        encoder = wide_resnet.__dict__[model_name](dropout_rate=0.0)
        hidden_dim = 128
    else:
        encoder = resnet.__dict__[model_name]()
        hidden_dim = 2048
        if 'w2' in model_name:
            hidden_dim *= 2
        elif 'w4' in model_name:
            hidden_dim *= 4

    # -- projection head
    encoder.fc = torch.nn.Sequential(OrderedDict([
        ('fc1', torch.nn.Linear(hidden_dim, hidden_dim)),
        ('bn1', torch.nn.BatchNorm1d(hidden_dim)),
        ('relu1', torch.nn.ReLU(inplace=True)),
        ('fc2', torch.nn.Linear(hidden_dim, hidden_dim)),
        ('bn2', torch.nn.BatchNorm1d(hidden_dim)),
        ('relu2', torch.nn.ReLU(inplace=True)),
        ('fc3', torch.nn.Linear(hidden_dim, output_dim))
    ]))

    # -- prediction head
    encoder.pred = None
    if use_pred:
        mx = 4  # 4x bottleneck prediction head
        pred_head = OrderedDict([])
        pred_head['bn1'] = torch.nn.BatchNorm1d(output_dim)
        pred_head['fc1'] = torch.nn.Linear(output_dim, output_dim//mx)
        pred_head['bn2'] = torch.nn.BatchNorm1d(output_dim//mx)
        pred_head['relu'] = torch.nn.ReLU(inplace=True)
        pred_head['fc2'] = torch.nn.Linear(output_dim//mx, output_dim)
        encoder.pred = torch.nn.Sequential(pred_head)

    encoder.to(device)
    logger.info(encoder)
    return encoder

In [21]:
def init_opt(
    encoder,
    iterations_per_epoch,
    start_lr,
    ref_lr,
    ref_mom,
    nesterov,
    warmup,
    num_epochs,
    weight_decay=1e-6,
    final_lr=0.0
):
    param_groups = [
        {'params': (p for n, p in encoder.named_parameters()
                    if ('bias' not in n) and ('bn' not in n))},
        {'params': (p for n, p in encoder.named_parameters()
                    if ('bias' in n) or ('bn' in n)),
         'LARS_exclude': True,
         'weight_decay': 0}
    ]
    optimizer = SGD(
        param_groups,
        weight_decay=weight_decay,
        momentum=0.9,
        nesterov=nesterov,
        lr=ref_lr)
    scheduler = WarmupCosineSchedule(
        optimizer,
        warmup_steps=warmup*iterations_per_epoch,
        start_lr=start_lr,
        ref_lr=ref_lr,
        final_lr=final_lr,
        T_max=num_epochs*iterations_per_epoch)
    optimizer = LARS(optimizer, trust_coefficient=0.001)
    return encoder, optimizer, scheduler

In [22]:
import os

# -- FOR DISTRIBUTED TRAINING ENSURE ONLY 1 DEVICE VISIBLE PER PROCESS
try:
    # -- WARNING: IF DOING DISTRIBUTED TRAINING ON A NON-SLURM CLUSTER, MAKE
    # --          SURE TO UPDATE THIS TO GET LOCAL-RANK ON NODE, OR ENSURE
    # --          THAT YOUR JOBS ARE LAUNCHED WITH ONLY 1 DEVICE VISIBLE
    # --          TO EACH PROCESS
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['SLURM_LOCALID']
except Exception:
    pass

import logging
import sys
from collections import OrderedDict

import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter
)
from src.losses import (
    init_paws_loss,
    make_labels_matrix
)
from src.data_manager import (
    init_data,
    make_transforms,
    make_multicrop_transform
)
from src.sgd import SGD
from src.lars import LARS

import apex
from torch.nn.parallel import DistributedDataParallel

# --
log_timings = True
log_freq = 10
checkpoint_freq = 50
# --

_GLOBAL_SEED = 0
np.random.seed(_GLOBAL_SEED)
torch.manual_seed(_GLOBAL_SEED)
torch.backends.cudnn.benchmark = True

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()

In [23]:
# ----------------------------------------------------------------------- #
#  PASSED IN PARAMS FROM CONFIG FILE
# ----------------------------------------------------------------------- #
# -- META
model_name = args['meta']['model_name']
output_dim = args['meta']['output_dim']
load_model = args['meta']['load_checkpoint']
r_file = args['meta']['read_checkpoint']
copy_data = args['meta']['copy_data']
use_fp16 = args['meta']['use_fp16']
use_pred_head = args['meta']['use_pred_head']
device = torch.device(args['meta']['device'])
torch.cuda.set_device(device)

# -- CRITERTION
reg = args['criterion']['me_max']
supervised_views = args['criterion']['supervised_views']
classes_per_batch = args['criterion']['classes_per_batch']
s_batch_size = args['criterion']['supervised_imgs_per_class']
u_batch_size = args['criterion']['unsupervised_batch_size']
temperature = args['criterion']['temperature']
sharpen = args['criterion']['sharpen']

# -- DATA
unlabeled_frac = args['data']['unlabeled_frac']
color_jitter = args['data']['color_jitter_strength']
normalize = args['data']['normalize']
root_path = args['data']['root_path']
image_folder = args['data']['image_folder']
dataset_name = args['data']['dataset']
subset_path = args['data']['subset_path']
unique_classes = args['data']['unique_classes_per_rank']
multicrop = args['data']['multicrop']
label_smoothing = args['data']['label_smoothing']
data_seed = None
if 'cifar10' in dataset_name:
    data_seed = args['data']['data_seed']
    crop_scale = (0.75, 1.0) if multicrop > 0 else (0.5, 1.0)
    mc_scale = (0.3, 0.75)
    mc_size = 18
else:
    crop_scale = (0.14, 1.0) if multicrop > 0 else (0.08, 1.0)
    mc_scale = (0.05, 0.14)
    mc_size = 96

In [24]:
world_size = 1
rank = 0 

In [25]:
# -- OPTIMIZATION
wd = float(args['optimization']['weight_decay'])
num_epochs = args['optimization']['epochs']
warmup = args['optimization']['warmup']
start_lr = args['optimization']['start_lr']
lr = args['optimization']['lr']
final_lr = args['optimization']['final_lr']
mom = args['optimization']['momentum']
nesterov = args['optimization']['nesterov']

# -- LOGGING
folder = args['logging']['folder']
tag = args['logging']['write_tag']

In [26]:
# -- log/checkpointing paths
log_file = os.path.join(folder, f'{tag}_r{rank}.csv')
save_path = os.path.join(folder, f'{tag}' + '-ep{epoch}.pth.tar')
latest_path = os.path.join(folder, f'{tag}-latest.pth.tar')
best_path = os.path.join(folder, f'{tag}' + '-best.pth.tar')
load_path = None
if load_model:
    load_path = os.path.join(folder, r_file) if r_file is not None else latest_path

In [27]:
# -- make csv_logger
csv_logger = CSVLogger(log_file,
                        ('%d', 'epoch'),
                        ('%d', 'itr'),
                        ('%.5f', 'paws-xent-loss'),
                        ('%.5f', 'paws-me_max-reg'),
                        ('%d', 'time (ms)'))

In [28]:
# -- init model
encoder = init_model(
    device=device,
    model_name=model_name,
    use_pred=use_pred_head,
    output_dim=output_dim)
if world_size > 1:
    process_group = apex.parallel.create_syncbn_process_group(0)
    encoder = apex.parallel.convert_syncbn_model(encoder, process_group=process_group)

INFO:root:ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), s

In [29]:
# -- init losses
paws = init_paws_loss(
    multicrop=multicrop,
    tau=temperature,
    T=sharpen,
    me_max=reg)

In [30]:
# -- assume support images are sampled with ClassStratifiedSampler
labels_matrix = make_labels_matrix(
    num_classes=classes_per_batch,
    s_batch_size=s_batch_size,
    world_size=world_size,
    device=device,
    unique_classes=unique_classes,
    smoothing=label_smoothing)

In [31]:
# -- make data transforms
transform, init_transform = make_transforms(
    dataset_name=dataset_name,
    subset_path=subset_path,
    unlabeled_frac=unlabeled_frac,
    training=True,
    split_seed=data_seed,
    crop_scale=crop_scale,
    basic_augmentations=False,
    color_jitter=color_jitter,
    normalize=normalize)
multicrop_transform = (multicrop, None)
if multicrop > 0:
    multicrop_transform = make_multicrop_transform(
            dataset_name=dataset_name,
            num_crops=multicrop,
            size=mc_size,
            crop_scale=mc_scale,
            normalize=normalize,
            color_distortion=color_jitter)

INFO:root:making imagenet data transforms
INFO:root:keep file: dr_subsets/90percent.txt


In [32]:
# -- init data-loaders/samplers
(unsupervised_loader,
    unsupervised_sampler,
    supervised_loader,
    supervised_sampler) = init_data(
        dataset_name=dataset_name,
        transform=transform,
        init_transform=init_transform,
        supervised_views=supervised_views,
        u_batch_size=u_batch_size,
        s_batch_size=s_batch_size,
        unique_classes=unique_classes,
        classes_per_batch=classes_per_batch,
        multicrop_transform=multicrop_transform,
        world_size=world_size,
        rank=rank,
        root_path=root_path,
        image_folder=image_folder,
        training=True,
        copy_data=copy_data)
iter_supervised = None
ipe = len(unsupervised_loader)
logger.info(f'iterations per epoch: {ipe}')

INFO:root:copying data locally
INFO:root:No job-id, will load directly from network file
INFO:root:data-path datasets/dr/sample@1000/train/
INFO:root:Initialized ImageDR
INFO:root:ImageDR dataset created
self.multicrop_transform (6, Compose(
    Compose(
    RandomResizedCrop(size=(96, 96), scale=(0.05, 0.14), ratio=(0.75, 1.3333), interpolation=bilinear)
    RandomHorizontalFlip(p=0.5)
    Compose(
    RandomApply(
    p=0.8
    ColorJitter(brightness=[0.19999999999999996, 1.8], contrast=[0.19999999999999996, 1.8], saturation=[0.19999999999999996, 1.8], hue=[-0.2, 0.2])
)
    RandomGrayscale(p=0.2)
)
    ToTensor()
)
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
))
INFO:root:ImageDR unsupervised data loader created
INFO:root:Making supervised ImageDR data loader...
self.multicrop_transform (0, None)
INFO:root:flipping coin to keep labels
INFO:root:supervised-reset-period 31
INFO:root:ImageDR supervised data loader created
INFO:root:iterations per epoch: 31


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [33]:
# -- init optimizer and scheduler
scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)
encoder, optimizer, scheduler = init_opt(
    encoder=encoder,
    weight_decay=wd,
    start_lr=start_lr,
    ref_lr=lr,
    final_lr=final_lr,
    ref_mom=mom,
    nesterov=nesterov,
    iterations_per_epoch=ipe,
    warmup=warmup,
    num_epochs=num_epochs)
if world_size > 1:
    encoder = DistributedDataParallel(encoder, broadcast_buffers=False)

In [34]:
start_epoch = 0
# -- load training checkpoint
if load_model:
    encoder, optimizer, start_epoch = load_checkpoint(
        r_path=load_path,
        encoder=encoder,
        opt=optimizer,
        scaler=scaler,
        use_fp16=use_fp16)
    for _ in range(start_epoch):
        for _ in range(ipe):
            scheduler.step()

In [35]:
##########################################
################## DEBUG #################
##########################################

In [36]:
epoch = 0 

In [37]:
# -- update distributed-data-loader epoch
unsupervised_sampler.set_epoch(epoch)
if supervised_sampler is not None:
    supervised_sampler.set_epoch(epoch)

loss_meter = AverageMeter()
ploss_meter = AverageMeter()
rloss_meter = AverageMeter()
time_meter = AverageMeter()
data_meter = AverageMeter()

In [38]:
best_loss = None

In [39]:
for itr, udata in enumerate(unsupervised_loader):

    def load_imgs():
        # -- unsupervised imgs
        uimgs = [u.to(device, non_blocking=True) for u in udata[:-1]]
        # -- supervised imgs
        global iter_supervised
        try:
            sdata = next(iter_supervised)
        except Exception:
            iter_supervised = iter(supervised_loader)
            logger.info(f'len.supervised_loader: {len(iter_supervised)}')
            sdata = next(iter_supervised)
        finally:
            labels = torch.cat([labels_matrix for _ in range(supervised_views)])
            simgs = [s.to(device, non_blocking=True) for s in sdata[:-1]]
        # -- concatenate supervised imgs and unsupervised imgs
        imgs = simgs + uimgs
        return imgs, labels
    (imgs, labels), dtime = gpu_timer(load_imgs)
    data_meter.update(dtime)
    break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


INFO:root:len.supervised_loader: 31


In [40]:
labels.shape

torch.Size([64, 2])

In [41]:
imgs[1].shape

torch.Size([32, 3, 224, 224])

In [42]:
imgs[8].shape

torch.Size([32, 3, 96, 96])

In [43]:
labels.shape

torch.Size([64, 2])

Cuantas simgs? y cuantas uimgs?

simages siempre van adelante tons

In [44]:
from logging import getLogger
import torch
from src.utils import (
    AllGather,
    AllReduce
)

In [45]:
tau=0.1
T=0.25
me_max=True

In [46]:
with torch.cuda.amp.autocast(enabled=use_fp16):
    optimizer.zero_grad()

    # --
    # h: representations of 'imgs' before head
    # z: representations of 'imgs' after head
    # -- If use_pred_head=False, then encoder.pred (prediction
    #    head) is None, and _forward_head just returns the
    #    identity, z=h

    """
    Nota: Creo que ya entiendo, las diferentes representaciones/vistas son realizadas es con esto 
          de antes de la cabeza y despues de la cabeza. 
    """

    h, z = encoder(imgs, return_before_head=True)

    # Compute paws loss in full precision
    with torch.cuda.amp.autocast(enabled=False):

        # Step 1. convert representations to fp32
        h, z = h.float(), z.float()

        # Step 2. determine anchor views/supports and their
        #         corresponding target views/supports
        # --

        """
        Nota: Este indice de `num_support` es muy importante ya que me dira hasta 
              donde seran las representaciones del anchor y desde donde 
              las etiquetadas (supports). 
        """

        num_support = supervised_views * s_batch_size * classes_per_batch
        # --
        anchor_supports = z[:num_support]
        anchor_views = z[num_support:]
        # --
        """
        Nota: Este `target_supportst` me dice cuales son las representaciones de `support` (las etiquetadas)
              que voy a utilizar con las vistas. 
        """
        target_supports = h[:num_support].detach() # Este Target es una REPRESENTACION
        target_views = h[num_support:].detach()
        target_views = torch.cat([
            target_views[u_batch_size:2*u_batch_size],
            target_views[:u_batch_size]], dim=0)

        softmax = torch.nn.Softmax(dim=1)

        def sharpen(p):
            sharp_p = p**(1./T)
            sharp_p /= torch.sum(sharp_p, dim=1, keepdim=True)
            return sharp_p

        def snn(query, supports, labels):
            """ Soft Nearest Neighbours similarity classifier """
            # Step 1: normalize embeddings
            query = torch.nn.functional.normalize(query)
            supports = torch.nn.functional.normalize(supports)

            # Step 2: gather embeddings from all workers
            supports = AllGather.apply(supports)

            # Step 3: compute similarlity between local embeddings
            return softmax(query @ supports.T / tau) @ labels


        """
        Nota: `anchor_support_labels` y `target_support_labels` son las mismas 
               las que vienen de `labels`. 
        """
        #anchor_views=anchor_views
        #anchor_supports=anchor_supports
        anchor_support_labels=labels
        #target_views=target_views
        #target_supports=target_supports
        target_support_labels=labels
        # -- NOTE: num views of each unlabeled instance = 2+multicrop
        batch_size = len(anchor_views) // (2+multicrop)

        # Step 1: compute anchor predictions
        probs = snn(anchor_views, anchor_supports, anchor_support_labels)


        # Step 2: compute targets for anchor predictions
        with torch.no_grad():
            targets = snn(target_views, target_supports, target_support_labels)
            targets = sharpen(targets)
            if multicrop > 0:
                mc_target = 0.5*(targets[:batch_size]+targets[batch_size:])
                targets = torch.cat([targets, *[mc_target for _ in range(multicrop)]], dim=0)
            targets[targets < 1e-4] *= 0  # numerical stability

        # Step 3: compute cross-entropy loss H(targets, queries)
        loss = torch.mean(torch.sum(torch.log(probs**(-targets)), dim=1))

        # Step 4: compute me-max regularizer
        rloss = 0.
        if me_max:
            avg_probs = AllReduce.apply(torch.mean(sharpen(probs), dim=0))
            rloss -= torch.sum(torch.log(avg_probs**(-avg_probs)))


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [47]:
rloss

tensor(-0.6930, device='cuda:0', grad_fn=<RsubBackward1>)

In [ ]:
########### DEBUG   2 ################

In [44]:
optimizer.zero_grad()

In [45]:
h, z = encoder(imgs, return_before_head=True)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [46]:
z.shape

torch.Size([320, 2048])

In [47]:
h.shape

torch.Size([320, 2048])

In [48]:
h, z = h.float(), z.float()

In [49]:
num_support = supervised_views * s_batch_size * classes_per_batch

In [50]:
anchor_supports = z[:num_support]

In [51]:
anchor_views = z[num_support:]

In [52]:
target_supports = h[:num_support].detach() # Este Target es una REPRESENTACION

In [53]:
target_views = h[num_support:].detach()

In [54]:
target_views = torch.cat([
    target_views[u_batch_size:2*u_batch_size],
    target_views[:u_batch_size]], dim=0)

softmax = torch.nn.Softmax(dim=1)

In [55]:
def sharpen(p):
    sharp_p = p**(1./T)
    sharp_p /= torch.sum(sharp_p, dim=1, keepdim=True)
    return sharp_p

def snn(query, supports, labels):
    """ Soft Nearest Neighbours similarity classifier """
    # Step 1: normalize embeddings
    query = torch.nn.functional.normalize(query)
    supports = torch.nn.functional.normalize(supports)

    # Step 2: gather embeddings from all workers
    supports = AllGather.apply(supports)

    # Step 3: compute similarlity between local embeddings
    return softmax(query @ supports.T / tau) @ labels

In [56]:
anchor_support_labels=labels

target_support_labels=labels

In [57]:
# -- NOTE: num views of each unlabeled instance = 2+multicrop
batch_size = len(anchor_views) // (2+multicrop)

In [58]:
# Step 1: compute anchor predictions
probs = snn(anchor_views, anchor_supports, anchor_support_labels)

In [59]:
# Step 2: compute targets for anchor predictions
with torch.no_grad():
    targets = snn(target_views, target_supports, target_support_labels)
    targets = sharpen(targets)
    if multicrop > 0:
        mc_target = 0.5*(targets[:batch_size]+targets[batch_size:])
        targets = torch.cat([targets, *[mc_target for _ in range(multicrop)]], dim=0)
    targets[targets < 1e-4] *= 0  # numerical stability

In [60]:
loss = torch.mean(torch.sum(torch.log(probs**(-targets)), dim=1))

In [61]:
# Step 4: compute me-max regularizer
rloss = 0.
if me_max:
    avg_probs = AllReduce.apply(torch.mean(sharpen(probs), dim=0))
    rloss -= torch.sum(torch.log(avg_probs**(-avg_probs)))

In [62]:
###############@###################